# XGBoost

## 1. Importación de librerías, establecimiento de directorio de trabajo y carga de datos

In [1]:
# librerias principales para uso y visualización de datos
import os
import numpy as np
import pandas as pd
import warnings  
warnings.filterwarnings('ignore')

# dividir datos para modelo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# algoritmo
from xgboost import XGBRegressor

# metricas de rendimiento
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

# guardar el modelo
import pickle

# para visualización
import seaborn as sns  
import matplotlib.pyplot as plt  
from mpl_toolkits.mplot3d import Axes3D  
%matplotlib inline  

os.chdir("C:/Users/aleja/OneDrive - Universidad Politécnica de Madrid/Universidad/Asignaturas/Cuarto/Trabajo de Fin de Grado")

In [2]:
accesos = pd.read_csv("Resultados/Accesos_Calidad_Vida.csv", index_col="Unnamed: 0")
accesos.head()

,latitud,longitud,edad,calidad_vida
0,41.603431,2.100698,10,28.268551
1,41.633288,2.062058,11,20.730271
2,41.633312,2.063174,12,20.730271
3,41.631481,2.063229,11,10.011779
4,41.632723,2.063037,14,20.730271


## 2. Construcción de la matriz de características (X) y el vector variable dependiente (y)

In [3]:
X = accesos.iloc[:, :-1].values
y = accesos.iloc[:, -1].values

## 3. División del dataset en set de entrenamiento y de test

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

## 4. Feature scaling of X (rango: -3 a 3)

In [5]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 5. Construcción y entrenamiento del modelo de regresión sobre el set de entrenamiento

In [6]:
regressor = XGBRegressor(n_estimators=700, max_depth=7, objective='reg:squarederror')
regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=700, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

## 6. Predicción de resultados del set de test

In [7]:
y_pred = regressor.predict(X_test)

## 7.  Métricas de rendimiento
Comparamos el set de test con el predicho

In [8]:
# R2score
print('R2score:', r2_score(y_test, y_pred))

# Mean Absolute Error
print('MAE: \t', mean_absolute_error(y_test, y_pred))

# Mean Squared Error
print('MSE: \t', mean_squared_error(y_test, y_pred))

# Root Mean Squared Error
print("RMSE: ", mean_squared_error(y_test, y_pred, squared=False))

R2score: 0.9980356076873104
MAE: 	 0.2689110502482483
MSE: 	 1.471074828483071
RMSE:  1.2128787360998095


### 7.1. Para ganar más intuición

In [9]:
np.set_printoptions(precision=2) # redondea a dos decimales

print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[ 6.80e+01  6.80e+01]
 [ 6.77e+01  6.80e+01]
 [ 6.80e+01  6.80e+01]
 ...
 [ 6.80e+01  6.80e+01]
 [-4.49e-03  0.00e+00]
 [ 6.80e+01  6.80e+01]]


### 7.2 Más métricas de rendimiento

In [10]:
accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)

print("Media: {:.2f} %".format(accuracies.mean()*100))
print("Desviación estándar: {:.2f} %".format(accuracies.std()*100))

Media: 99.78 %
Desviación estándar: 0.04 %


Vemos que la precision y los errores no son muy buenos, por lo que probaremos otros parámetros para ver si ofrece alguno más óptimo
### 7.3 Grid Search

In [11]:
parameters = {
    'n_estimators': [700],
    'max_depth':[7]
}

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parameters,
                           scoring = 'r2',
                           cv = cv,
                           n_jobs = -1)

grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print("Best r2: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Best r2: 99.78 %
Best Parameters: {'max_depth': 7, 'n_estimators': 700}


## 8. Guardar el mejor modelo

In [12]:
with open('Modelos/XGBoost.pkl','wb') as f:
    pickle.dump(regressor, f)

## 9. Ejemplo de predicción

In [13]:
print(regressor.predict(sc.transform([['425060.955125', '4.606138e+06', '22']]))) 

[-0.13]
